In [133]:
import pandas as pd
from sklearn.linear_model import LogisticRegression

In [95]:
database = pd.read_csv('COBRA-YTD2017.csv')

In [112]:
database.rename(columns={'UC2 Literal' : 'Crime', 'occur_time': 'time', 'MaxOfnum_victims': 'victims'}, inplace=True)
display(database.head)

<bound method NDFrame.head of        MI_PRINX  offense_id    rpt_date  occur_date                time  \
0       8924155   173650072  12/31/2017  12/30/2017 2018-04-13 23:15:00   
1       8924156   173650102  12/31/2017  12/18/2017 2018-04-13 13:00:00   
2       8924157   173650144  12/31/2017  12/30/2017 2018-04-13 22:01:00   
3       8924158   173650149  12/31/2017  12/30/2017 2018-04-13 20:00:00   
4       8924159   173650159  12/31/2017  12/31/2017 2018-04-13 00:41:00   
5       8924160   173650180  12/31/2017  12/30/2017 2018-04-13 23:00:00   
6       8924161   173650236  12/31/2017  12/31/2017 2018-04-13 01:55:00   
7       8924162   173650241  12/31/2017  12/31/2017 2018-04-13 00:00:00   
8       8924163   173650295  12/31/2017  12/30/2017 2018-04-13 00:00:00   
9       8924164   173650389  12/31/2017  12/31/2017 2018-04-13 00:00:00   
10      8924165   173650449  12/31/2017  12/31/2017 2018-04-13 00:40:00   
11      8924166   173650562  12/31/2017  12/31/2017 2018-04-13 00:00:0

In [114]:
database.Crime.unique()

array(['LARCENY-FROM VEHICLE', 'ROBBERY-PEDESTRIAN', 'BURGLARY-RESIDENCE',
       'AUTO THEFT', 'BURGLARY-NONRES', 'ROBBERY-RESIDENCE',
       'LARCENY-NON VEHICLE', 'AGG ASSAULT', 'RAPE', 'ROBBERY-COMMERCIAL',
       'HOMICIDE'], dtype=object)

In [115]:
# NOTE: Clean up database.
database['Crime'] = database['Crime'].replace({'LARCENY-FROM VEHICLE': '1',
                          'ROBBERY-PEDESTRIAN': '2',
                          'BURGLARY-RESIDENCE': '3',
                          'AUTO THEFT': '4',
                          'BURGLARY-NONRESIDENCE': '5',
                          'BURGLARY-NONRES': '6',
                          'ROBBERY-RESIDENCE': '7',
                          'LARCENY-NON VEHICLE': '8',
                          'AGG ASSAULT': '9',
                          'ROBBERY-COMMERCIAL': '10',
                          'HOMICIDE': '11'})

In [156]:
# database['time'] = pd.to_datetime(database.time)
# # TODO: Figure out how to plot a scatter_matrix

# # from pandas.plotting import scatter_matrix

# # database['occur_time'] = pd.to_datetime(database['occur_time'], errors='coerce')

# # database['occur_time'] = database['occur_time'].dt.time

# database['hour'] = database.time.dt.hour
# # database['hour'].value_counts()

database['victims'] = database['victims'].fillna(0)
database.victims = database.victims.astype(int)
database.hour = database.victims.astype(int)
database.x = database.x.astype(int)
database.y = database.y.astype(int)

database.dtypes

MI_PRINX                      int64
offense_id                    int64
rpt_date                     object
occur_date                   object
time                 datetime64[ns]
poss_date                    object
poss_time                    object
beat                          int64
apt_office_prefix            object
apt_office_num               object
location                     object
MinOfucr                      int64
MinOfibr_code                object
dispo_code                   object
victims                       int32
Shift                        object
Avg Day                      object
loc_type                    float64
Crime                        object
neighborhood                 object
npu                          object
x                             int32
y                             int32
hour                          int32
dtype: object

In [157]:
from sklearn.preprocessing import scale

X_all = database.drop(['Crime'], 1)
y_all = database['Crime']

X_all = database[['hour', 'victims', 'x', 'y']]

In [158]:
from sklearn.cross_validation import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, test_size = 50, random_state = 2, stratify = y_all)

X_train

,hour,victims,x,y
24995,1,1,-84,33
23425,3,3,-84,33
6324,1,1,-84,33
21330,1,1,-84,33
22965,2,2,-84,33
24783,1,1,-84,33
2267,1,1,-84,33
8933,1,1,-84,33
7683,1,1,-84,33
21534,1,1,-84,33


In [164]:
from sklearn.metrics import f1_score

def train_classifier(clf, X_train, y_train):
    clf.fit(X_train, y_train)
    
def predict_labels(clf, features, target):
    y_pred = clf.predict(features)
    
    return f1_score(target, y_pred, pos_label='1', average='macro'), sum(target == y_pred) / float(len(y_pred))

def train_predict(clf, X_train, y_train, X_test, y_test):
    
    train_classifier(clf, X_train, y_train)
    
    f1, acc = predict_labels(clf, X_train, y_train)
    
    f1, acc = predict_labels(clf, X_test, y_test)

In [165]:
clf_A = LogisticRegression(random_state = 42)

train_predict(clf_A, X_train, y_train, X_test, y_test)

c:\users\kbantupa\documents\github\atlanta_crime\env\lib\site-packages\sklearn\metrics\classification.py:1045: UserWarning: Note that pos_label (set to '1') is ignored when average != 'binary' (got 'macro'). You may use labels=[pos_label] to specify a single positive class.
  % (pos_label, average), UserWarning)
c:\users\kbantupa\documents\github\atlanta_crime\env\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [ ]:
n_matches = database.shape[0]

n_features = database.shape[1] - 1

n_larcfromvehicle = len(database[database.Crime == 'LARCENY-FROM VEHICLE'])
n_robberyped = len(database[database.Crime == 'ROBBERY-PEDESTRIAN'])

win_rate = (float(n_homewins) / (n_matches)) * 100

print(win_rate)